In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import imageio
import moviepy.editor as mpy
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warnings and info
SAMPLE_COL = 16
SAMPLE_ROW = 16
SAMPLE_NUM = SAMPLE_COL * SAMPLE_ROW

IMG_H = 64
IMG_W = 64
IMG_C = 3
IMG_SHAPE = (IMG_H, IMG_W, IMG_C)

BATCH_SIZE = 128
Z_DIM = 100
BZ = (BATCH_SIZE, Z_DIM)
BUF = 65536


W_LR_D = 3e-4
W_LR_G = 1.5e-4
W_EPOCH = 300

BUFFER_SIZE = 15000

2022-01-25 16:03:40.514630: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5111:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5111:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5111:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5599:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5111:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib co

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_visible_devices(gpus[3], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2022-01-25 16:03:42.100378: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


4 Physical GPUs, 1 Logical GPUs


2022-01-25 16:03:42.101575: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-25 16:03:42.129708: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-01-25 16:03:42.130766: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:82:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-01-25 16:03:42.131487: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:85:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88G

## Read Data

In [3]:
import os
file_list = os.listdir('./img_align_celeba_png/')
print(len(file_list))
print(file_list[0])
print('./img_align_celeba_png/%s' % file_list[0])

202599
000001.png
./img_align_celeba_png/000001.png


In [4]:
# Load the numpy files
def map_func(img_name):
    img = tf.io.read_file('./img_align_celeba_png/' + img_name)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    img = tf.image.resize(img, (IMG_H, IMG_W))
    img = (img / 255) # change range 
    return img

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((file_list))

# Use map to load the numpy files in parallel
dataset = dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Utility

In [6]:
# Utility function
def utPuzzle(imgs, row, col, path=None):
    h, w, c = imgs[0].shape
    out = np.zeros((h * row, w * col, c), np.uint8)
    for n, img in enumerate(imgs):
        j, i = divmod(n, col)
        out[j * h : (j + 1) * h, i * w : (i + 1) * w, :] = img
#     if path is not None : 
    imageio.imwrite(path, out)
    return out
  
def utMakeGif(imgs, fname, duration):
    n = float(len(imgs)) / duration
    clip = mpy.VideoClip(lambda t : imgs[int(n * t)], duration = duration)
    clip.write_gif(fname, fps = n)

## GAN

In [7]:
def GAN(img_shape, z_dim):
    # x-shape
    xh, xw, xc = img_shape
    # z-shape
    zh = xh // 4
    zw = xw // 4
        
    # return Generator and Discriminator
    return keras.Sequential([ # Generator
        keras.layers.Dense(units  =  1024, input_shape = (z_dim,)),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Dense(units  =  zh * zw << 8), # zh * zw * 256
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Reshape(target_shape = (zh, zw, 256)),
        keras.layers.Conv2DTranspose(
            filters = 32,
            kernel_size = 5,
            strides = 2,
            padding = "SAME"
        ),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Conv2DTranspose(
            filters = xc,
            kernel_size = 5,
            strides = 2,
            padding = "SAME",
            activation = keras.activations.sigmoid
        ),
    ]), keras.Sequential([ # Discriminator
        keras.layers.Conv2D(
            filters = 32,
            kernel_size = 5,
            strides = (2, 2),
            padding = "SAME",
            input_shape = img_shape,
        ),
        keras.layers.LeakyReLU(),
        keras.layers.Conv2D(
            filters = 128,
            kernel_size = 5,
            strides = (2, 2),
            padding = "SAME"
        ),
        keras.layers.LeakyReLU(),
        keras.layers.Flatten(),
        keras.layers.Dense(units  =  1024),
        keras.layers.LeakyReLU(),
        keras.layers.Dense(units  =  1),
    ])

s = tf.random.normal([SAMPLE_NUM, Z_DIM])

In [8]:
WG, WD = GAN(IMG_SHAPE, Z_DIM)
optimizer_g = keras.optimizers.Adam(W_LR_G, beta_1=0., beta_2=0.9)
optimizer_d = keras.optimizers.Adam(W_LR_D, beta_1=0., beta_2=0.9)


ckpt = tf.train.Checkpoint(WG_net=WG, WD_net=WD, optimizer_g=optimizer_g, optimizer_d=optimizer_d)

manager = tf.train.CheckpointManager(ckpt, './ckpts', max_to_keep=10,
                                     checkpoint_name='WGAN')
# ckpt.restore('./ckpts/WGAN-1')

@tf.function
def WGTrain(c1):
    z = tf.random.normal(BZ)
    with tf.GradientTape() as tpd:
        
        fake_img = WG(z, training = True)
        real_img = c1
        epsilon = tf.random.uniform((BATCH_SIZE, 1, 1, 1), minval=0, maxval=1, dtype=tf.dtypes.float32)
        c2 = epsilon * real_img + (1-epsilon) * fake_img

        z1 = WD(real_img, training = True)
        z0 = WD(fake_img, training = True)
        with tf.GradientTape() as tpd2:
            tpd2.watch(c2)
            z2 = WD(c2, training = True)
            

        gradient_c2 = tpd2.gradient(z2, c2)
        gradient_norm =tf.math.sqrt(tf.reduce_sum(tf.square(gradient_c2), axis=(1, 2, 3)))
        gradient_penalty = tf.reduce_mean(tf.square(gradient_norm - 1.0))
        
        lg = - tf.reduce_mean(z0)
        ld = tf.reduce_mean(z0) - tf.reduce_mean(z1) \
         + 10 * gradient_penalty

    gradient_g = tpd.gradient(lg, WG.trainable_variables)

    optimizer_g.apply_gradients(zip(gradient_g, WG.trainable_variables))
    
    return lg, ld

@tf.function
def WDTrain(c1):
    z = tf.random.normal(BZ)
    with tf.GradientTape() as tpd:
        
        fake_img = WG(z, training = True)
        real_img = c1
        epsilon = tf.random.uniform((BATCH_SIZE, 1, 1, 1), minval=0, maxval=1, dtype=tf.dtypes.float32)
        c2 = epsilon * real_img + (1-epsilon) * fake_img

        z1 = WD(real_img, training = True)
        z0 = WD(fake_img, training = True)
        with tf.GradientTape() as tpd2:
            tpd2.watch(c2)
            z2 = WD(c2, training = True)
            

        
        gradient_c2 = tpd2.gradient(z2, c2)
        gradient_norm = tf.math.sqrt(tf.reduce_sum(tf.square(gradient_c2), axis=(1, 2, 3)))
        gradient_penalty = tf.reduce_mean(tf.square(gradient_norm - 1.0))
        
        lg = - tf.reduce_mean(z0)
        ld = tf.reduce_mean(z0) - tf.reduce_mean(z1) \
         + 10 * gradient_penalty
                
    gradient_d = tpd.gradient(ld, WD.trainable_variables)

    optimizer_d.apply_gradients(zip(gradient_d, WD.trainable_variables))
    
    return lg, ld

In [9]:
WTrain = (
    WDTrain,
    WDTrain,
    WDTrain,
    WDTrain,
    WDTrain,
    WGTrain
)

WCritic = len(WTrain)

In [ ]:
wlg = [None] * W_EPOCH #record loss of g for each epoch
wld = [None] * W_EPOCH #record loss of d for each epoch
wsp = [None] * W_EPOCH #record sample images for each epoch

rsTrain = float(BATCH_SIZE) / float(len(file_list))
ctr = 0
for ep in range(W_EPOCH):
    lgt = 0.0
    ldt = 0.0
    for c1 in dataset:
        if(c1.shape[0] == BATCH_SIZE):
            lg, ld = WTrain[ctr](c1)
            ctr += 1
            lgt += lg.numpy()
            ldt += ld.numpy()
            if ctr == WCritic : ctr = 0
    wlg[ep] = lgt * rsTrain
    wld[ep] = ldt * rsTrain
    print("Epoch %d:" % ep)
    print("generator loss %3f " % wlg[ep])
    print("disciminator loss %3f" % wld[ep])
    print("\n")
    out = WG(s, training = False)
    img = utPuzzle(
        (out * 255.0).numpy().astype(np.uint8),
        SAMPLE_COL,
        SAMPLE_ROW,
        "imgs/w_%04d.png" % ep
    )
    wsp[ep] = img

2022-01-25 16:03:43.735401: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-25 16:03:43.753365: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000890000 Hz
2022-01-25 16:03:49.034347: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-25 16:03:49.887465: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-25 16:03:50.175004: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


Epoch 0:
generator loss -41.078419 
disciminator loss -11.376164


Epoch 1:
generator loss -32.170516 
disciminator loss -3.078866


Epoch 2:
generator loss -15.316301 
disciminator loss -2.904508


Epoch 3:
generator loss -7.861407 
disciminator loss -3.073332


Epoch 4:
generator loss -3.165942 
disciminator loss -3.169383


Epoch 5:
generator loss -1.348848 
disciminator loss -3.186712


Epoch 6:
generator loss -0.913736 
disciminator loss -3.117279


Epoch 7:
generator loss -0.358197 
disciminator loss -3.023639


Epoch 8:
generator loss -0.388839 
disciminator loss -2.941302


Epoch 9:
generator loss -0.694144 
disciminator loss -2.866858


Epoch 10:
generator loss -0.797079 
disciminator loss -2.797950


Epoch 11:
generator loss -0.714354 
disciminator loss -2.729118


Epoch 12:
generator loss -0.715142 
disciminator loss -2.673888


Epoch 13:
generator loss -0.577154 
disciminator loss -2.622777


Epoch 14:
generator loss -0.646519 
disciminator loss -2.575926


Epoch 15:
genera

Epoch 124:
generator loss -3.352531 
disciminator loss -1.259592


Epoch 125:
generator loss -3.262145 
disciminator loss -1.253771


Epoch 126:
generator loss -3.394707 
disciminator loss -1.252502


Epoch 127:
generator loss -3.189027 
disciminator loss -1.243811


Epoch 128:
generator loss -3.296335 
disciminator loss -1.246486


Epoch 129:
generator loss -3.219790 
disciminator loss -1.244708


Epoch 130:
generator loss -3.216094 
disciminator loss -1.236651


Epoch 131:
generator loss -3.172768 
disciminator loss -1.227468


Epoch 132:
generator loss -3.222558 
disciminator loss -1.226148


Epoch 133:
generator loss -3.322857 
disciminator loss -1.231474


Epoch 134:
generator loss -3.313283 
disciminator loss -1.221389


Epoch 135:
generator loss -3.250246 
disciminator loss -1.221906


Epoch 136:
generator loss -3.394973 
disciminator loss -1.215354


Epoch 137:
generator loss -3.312920 
disciminator loss -1.212307


Epoch 138:
generator loss -3.322819 
disciminator loss -1.2091

Epoch 247:
generator loss -3.994723 
disciminator loss -0.998052


Epoch 248:
generator loss -4.087296 
disciminator loss -0.993855


Epoch 249:
generator loss -4.107032 
disciminator loss -0.996128


Epoch 250:
generator loss -4.036721 
disciminator loss -0.991572


Epoch 251:
generator loss -4.112561 
disciminator loss -0.991716


Epoch 252:
generator loss -4.006261 
disciminator loss -0.989514


Epoch 253:
generator loss -3.926241 
disciminator loss -0.987627


Epoch 254:
generator loss -4.093718 
disciminator loss -0.987920


Epoch 255:
generator loss -3.953583 
disciminator loss -0.986394


Epoch 256:
generator loss -4.215789 
disciminator loss -0.985449


Epoch 257:
generator loss -4.210157 
disciminator loss -0.986635


Epoch 258:
generator loss -4.019529 
disciminator loss -0.981370


Epoch 259:
generator loss -4.072939 
disciminator loss -0.985502


Epoch 260:
generator loss -4.107701 
disciminator loss -0.984343


Epoch 261:
generator loss -4.118053 
disciminator loss -0.9801

In [ ]:
manager.save()

In [ ]:
utMakeGif(np.array(wsp), "imgs/wgan.gif", duration = 2)

In [ ]:
plt.plot(range(W_EPOCH), wld, color = "blue", label = "Discriminator Loss")
plt.plot(range(W_EPOCH), wlg, color = "red", label = "Generator Loss")
plt.legend(loc = "upper right")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("WGAN Training Loss")
plt.show()